In [46]:
import cmapPy
import cmapPy.pandasGEXpress.parse
import json
import numpy as np
from collections import Counter
np.random.seed(1024)
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels
from statsmodels.stats.multitest import fdrcorrection
import os

# CMAP2 Enrichment Score Calculation (one sample, one cell type)

In [ ]:
# rank file of 6hr perturbations (from z scores)- gene x compound effects
cmap2_cmap1_perturb_rank_dur = pd.read_csv('/share/fsmresfiles/SpatialT/cmap2_l1000/processed/6hr_only/cmap2_6hr_pert_rank.csv', engine='c')

In [22]:
cmap2_cmap1_perturb_rank_dur = cmap2_cmap1_perturb_rank_dur.set_index('Unnamed: 0')

In [23]:
cmap2_cmap1_perturb_rank_dur

,BIX-01338--LPROT001_A375_6H:BRD-K26863634:10,compound-e--LPROT002_A375_6H:BRD-A55453956:0.1,UNC-1215--LPROT004_YAPC_6H:BRD-K83189926:1,BAFILOMYCIN-A1--LPROT002_NPC.TAK_6H:BRD-K86787118:0.05,gatifloxacin--CPD003_MCF7_6H:BRD-A74980173-001-08-5:10,alfaxalone--CPD003_PC3_6H:J05,camptothecin--RAD001_MCF7_6H:BRD-K37890730-001-10-2:10,BRD-A72498772--AML001_PC3_6H:BRD-A72498772:0.37037,losmapimod--LPROT005_A549_6H:BRD-K56195681:0.25,BRD-K54343811--HDAC002_MCF7_6H:BRD-K54343811-001-01-1:10,...,bisphenol-a--CRCGN004_MCF7_6H:BRD-K41494493-001-07-0:10,DMSO--CRCGN001_HA1E_6H:A17,BRD-K45330754--CRCGN004_PC3_6H:BRD-K45330754-001-12-6:19.9,BRD-A21490874--CRCGN004_HA1E_6H:BRD-A21490874-001-01-6:20.1,estriol--CRCGN004_HA1E_6H:BRD-A88939772-001-06-4:20.1,ifosfamide--CRCGN004_HA1E_6H:BRD-A67097164-001-14-6:20,methyl-testosterone--CRCGN004_MCF7_6H:BRD-A08062015-001-01-0:20.8,busulfan--CRCGN004_MCF7_6H:BRD-K23204545-001-12-3:20.1,urethane--CRCGN001_HA1E_6H:BRD-K24297741-001-04-5:21.1,BRD-K45330754--CRCGN004_HMELZ_6H:BRD-K45330754-001-12-6:19.9
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
NAT2,4651.0,2927.0,9537.0,5050.0,3800.0,3178.0,3044.0,8619.0,7870.0,10370.0,...,2865.5,15.0,11721.0,8159.0,9226.5,7663.0,4462.0,8752.0,7550.0,7694.5
ADA,330.0,411.0,7649.0,10124.0,457.0,7751.0,6530.0,8859.0,6031.0,6637.0,...,9048.5,10124.0,10555.0,4379.0,1999.0,11534.0,6175.0,11822.0,1098.0,5959.5
CDH2,12016.0,8905.0,9842.0,8425.0,8518.0,2350.0,3693.0,10627.0,5207.0,3180.0,...,8549.0,11179.0,442.0,8128.0,11713.5,3584.0,11702.5,1711.5,9635.0,1380.0
AKT3,4199.0,501.0,7001.0,7604.0,115.0,9042.0,7115.0,1558.0,8258.0,6084.0,...,5842.5,7016.5,7098.5,6378.0,3980.0,1981.0,1580.0,10922.5,3071.0,5335.0
MED6,7920.0,6174.0,9731.0,9954.0,11881.0,7893.0,584.0,617.0,4985.0,1070.0,...,10262.0,11872.0,2524.0,9461.0,11729.0,2884.0,10362.0,2470.0,8440.0,485.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SLC12A6,3062.0,8020.0,10882.0,5372.0,5888.5,11173.0,10310.0,3801.0,5393.0,6106.0,...,7946.0,7903.0,12259.0,5645.0,8102.0,9623.5,300.0,8859.0,3375.0,3495.0
PTBP3,5066.0,1383.0,9788.0,12276.0,11791.0,2061.5,4224.5,2917.5,2242.0,4499.5,...,8055.5,640.0,8008.0,9323.0,12103.0,3222.0,12182.0,5380.5,3627.0,1035.0
KCNE2,1725.0,6056.0,11588.0,922.0,1281.0,6226.0,8829.0,11037.0,11155.0,7069.5,...,3027.0,969.5,10788.5,2309.5,941.0,7276.5,980.0,4812.5,188.0,11257.5


In [29]:
cmap2_cmap1_perturb_rank_dur_arry = np.array(cmap2_cmap1_perturb_rank_dur)

## Format DGE results to get only up and down DGEs

In [116]:
# identify cell type of interest, in previous iteration 4 cell types are tested, hence some downstream results show 4 cell types
# only endo is shown for demo purpose
endo = pd.read_csv('/share/fsmresfiles/SpatialT/drug-target/DS4A/DS4A.1/DGE_dec/Endothelial.txt',sep='\t')

In [9]:
def get_up_down_deg (df):
    df_up = df[(df['stat']>0.5) & (df['qval']<0.05) ]
    df_down = df[(df['stat']<-0.5) & (df['qval']<0.05) ]
    print(df_up.shape, df_down.shape)
    return df_up, df_down

In [117]:
endo_up, endo_down = get_up_down_deg(endo)

(649, 4) (636, 4)
(270, 4) (693, 4)
(578, 4) (741, 4)
(395, 4) (1261, 4)


In [11]:
hgnc_v2 = pd.read_csv('/share/fsmresfiles/SpatialT/drug-target-results-OVR-0725/Analysis_PPI/HGNC091923.txt', sep='\t')

In [12]:
hgnc_dict = dict(zip(hgnc_v2['Approved symbol'], hgnc_v2['NCBI gene ID']))
hgnc_v2_prev = hgnc_v2[hgnc_v2['Previous symbol'].notnull()]
hgnc_dict_prev = dict(zip(hgnc_v2_prev['Previous symbol'], hgnc_v2_prev['NCBI gene ID']))

In [13]:
# transform from genename to number
def gene_up_down(up_df, down_df):
    gene_num_up = []
    for i in up_df['gene'].tolist():
        if i in hgnc_dict.keys():
            gene_num_up.append(str(int(hgnc_dict[i])))
        elif i in hgnc_dict_prev.keys():
            gene_num_up.append(str(int(hgnc_dict_prev[i])))
        else:
            print(i)

    gene_num_down = []
    for i in down_df['gene'].tolist():
        if i in hgnc_dict.keys():
            gene_num_down.append(str(int(hgnc_dict[i])))
        elif i in hgnc_dict_prev.keys():
            gene_num_down.append(str(int(hgnc_dict_prev[i])))
        else:
            print(i)
    print(len(gene_num_up), len(gene_num_down))        
    return gene_num_up, gene_num_down

In [118]:
gene_num_up_endo, gene_num_down_endo = gene_up_down(endo_up, endo_down )

649 636
270 693
578 741
395 1261


In [15]:
def write_gene_up_down(gene_lst, file_name):
    with open('/share/fsmresfiles/SpatialT/drug-target-results-OVR-0725/Analysis_DEG/'+file_name+'.txt', 'w') as f:
        for line in gene_lst:
            f.write(f"{line}\n")

In [18]:
write_gene_up_down(gene_num_up_endo, 'ds4a_endo_up_deg')
write_gene_up_down(gene_num_down_endo, 'ds4a_endo_down_deg')

In [19]:
gene_info = pd.read_csv('/share/fsmresfiles/SpatialT/cmap2_l1000/geneinfo_beta.txt', sep='\t')

In [20]:
# mapping gene indices to real names 
gene_cmap2 = gene_info[['gene_id','gene_symbol']]
gene_cmap2.columns = ['num','name']
lst = []
for i in gene_cmap2['num'].tolist():
    if pd.isna(i) != True:
        lst.append(int(i))
    else:
        lst.append(i)
gene_cmap2['num'] = lst
gene_dict_cmap2 = gene_cmap2.set_index('num').to_dict()['name']
gene_dict_cmap2_rev = {v:k for k,v in gene_dict_cmap2.items()}

/tmp/ipykernel_36061/3714792454.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_cmap2['num'] = lst


## GSEA

In [24]:
ROWS_dur = [str(gene_dict_cmap2_rev[i]) for i in cmap2_cmap1_perturb_rank_dur.index]
COLS_dur = cmap2_cmap1_perturb_rank_dur.columns.tolist()

In [25]:
ROW2IDX2_val_dur = list(range(0,len(ROWS_dur)))
COL2IDX2_val_dur = list(range(0,len(COLS_dur)))

In [26]:
ROW2IDX_key_dur = ROWS_dur
COL2IDX_key_dur = COLS_dur

In [27]:
ROW2IDX_dur = {k:v for k,v in zip(ROW2IDX_key_dur, ROW2IDX2_val_dur)}
COL2IDX_dur = {k:v for k,v in zip(COL2IDX_key_dur, COL2IDX2_val_dur)}

In [30]:
N_GENES, N_PERTURBATIONS = cmap2_cmap1_perturb_rank_dur_arry.shape
N_REPEATS = 1000
RANDOM_RANK = np.random.rand(N_GENES, N_REPEATS).argsort(axis=0) + 1  # gene x repeat
    
def cmap2_gsea_setup(gene_num_up, gene_num_down):
    DATA = {
        "U": [ROW2IDX_dur[g] for g in gene_num_up if g in ROW2IDX_dur],
        "D": [ROW2IDX_dur[g] for g in gene_num_down if g in ROW2IDX_dur],
    }
    N_UP = len(DATA["U"])
    N_DOWN = len(DATA["D"])
    print(N_UP, N_DOWN)
    return DATA, N_UP, N_DOWN

In [31]:
DATA_ENDO, N_UP_ENDO, N_DOWN_ENDO = cmap2_gsea_setup(gene_num_up_endo, gene_num_down_endo )

547 540
225 595
480 640


In [32]:
# up/down gene set level es
def STEP1(v):
    # deg sorted up/down reg gene perrturb array
    v = sorted(v)
    #print('v:', v, len(v))
    # t = length of up/down reg gene perrturb array
    t = len(v)
    
    # (i+1)/t = fraction of elements up to index i in sorted up/down gene list 
    # v[i]/n_genes = fraction of the rank of the current gene at index i out of the total number of genes
    # total upward (a) and downward (b) influence (es) for each up/down reg gene set
    a = max((i + 1) / t - v[i] / N_GENES for i in range(t)) #look into paper percentage versus ngenes (window)
    b = max(v[i] / N_GENES - i / t for i in range(t))
    es = 0
    if a > b: #check what a and b is
        es = a
    elif b > a:
        es = -b
    #print('a:',a,'b:', b)
    return es

# diff of up/down gene set es 
def STEP2(rank_u, rank_d):
    #print(rank_u)
    # calculate up genes and down genes es difference for each perturbation to determine overall es  
    es_u = STEP1(rank_u) if rank_u.size else 0
    es_d = STEP1(rank_d) if rank_d.size else 0
    #print(es_u, es_d)
    if np.sign(es_u) == np.sign(es_d):
        return 0
    return es_u - es_d


In [33]:
def run_gsea(DATA, N_UP, N_DOWN):
    background = np.array([STEP2(RANDOM_RANK[:N_UP, c], RANDOM_RANK[N_UP:N_UP + N_DOWN, c]) for c in range(N_REPEATS)])
    result = []
    # for each perturbation, calculate the es score difference of up genes and down genes
    for col in range(N_PERTURBATIONS):
        es = STEP2(cmap2_cmap1_perturb_rank_dur_arry[DATA["U"], col], cmap2_cmap1_perturb_rank_dur_arry[DATA["D"], col])
        print(es)
        p = 1.0
        if es > 0:
            p = np.mean(background > es)
        elif es < 0:
            p = np.mean(background < es)
        result.append((es, p))
    return result

In [ ]:
result_endo= run_gsea(DATA_ENDO, N_UP_ENDO, N_DOWN_ENDO)

In [40]:
def format_gsea_res(result):
    res_formatted = pd.DataFrame(result)
    res_formatted.columns = ['es','p']
    res_formatted['es'] = [round(i,3) for i in res_formatted['es'].tolist()]
    res_formatted['p-adj']=statsmodels.stats.multitest.fdrcorrection(res_formatted['p'].tolist(), alpha=0.05, method='indep', is_sorted=False)[1]
    res_formatted = res_formatted.reset_index()
    res_formatted.index = COLS_dur
    #res_formatted = res_formatted.drop('level_0',axis = 1)
    res_formatted['drug'] = [i.split('--')[0] for i in res_formatted.index.tolist()]
    return res_formatted

In [41]:
result_endo_form = format_gsea_res(result_endo)

In [42]:
result_endo_form.to_csv('/share/fsmresfiles/SpatialT/cmap2_l1000/processed/6hr_only/enrich_ds4a/ds4a_cmap2_6hr_endo_enrich.csv')

In [48]:
os.chdir('/share/fsmresfiles/SpatialT/cmap2_l1000/processed/6hr_only/enrich_ds4a/')

In [49]:
# below result is for fibroblasts, but endo should follow the same format
# result_fib_form

,index,es,p,p-adj,drug
BIX-01338--LPROT001_A375_6H:BRD-K26863634:10,0,0.231,0.000,0.000000,BIX-01338
compound-e--LPROT002_A375_6H:BRD-A55453956:0.1,1,0.000,1.000,1.000000,compound-e
UNC-1215--LPROT004_YAPC_6H:BRD-K83189926:1,2,0.119,0.001,0.003514,UNC-1215
BAFILOMYCIN-A1--LPROT002_NPC.TAK_6H:BRD-K86787118:0.05,3,0.000,1.000,1.000000,BAFILOMYCIN-A1
gatifloxacin--CPD003_MCF7_6H:BRD-A74980173-001-08-5:10,4,0.000,1.000,1.000000,gatifloxacin
...,...,...,...,...,...
ifosfamide--CRCGN004_HA1E_6H:BRD-A67097164-001-14-6:20,145486,-0.153,0.000,0.000000,ifosfamide
methyl-testosterone--CRCGN004_MCF7_6H:BRD-A08062015-001-01-0:20.8,145487,0.000,1.000,1.000000,methyl-testosterone
busulfan--CRCGN004_MCF7_6H:BRD-K23204545-001-12-3:20.1,145488,0.000,1.000,1.000000,busulfan
urethane--CRCGN001_HA1E_6H:BRD-K24297741-001-04-5:21.1,145489,0.000,1.000,1.000000,urethane


## Format GSEA result for drug enrichment output

In [76]:
def format_enrich(file_name, cell):
    ds = pd.read_csv(file_name)
    res = pd.DataFrame()
    res['CMAP_instance'] = [i.split('--')[1] for i in ds['Unnamed: 0'].tolist()]
    res['Drug'] = ds['drug']
    res['Enrichment_score'] = ds['es']
    res['P-value'] = ds['p']
    res['Adj-p']= ds['p-adj']
    res = res.sort_values('Enrichment_score', ascending=False)
    res_inv = res.head(200)
    res_pos = res.tail(200)
    res_inv.to_csv('/share/fsmresfiles/SpatialT/website/staging/data_v3_release/drug_screen/drug_enrichment/DS4A/DS4A.1/'+cell+'_INV_es.csv', index=False)
    res_pos.to_csv('/share/fsmresfiles/SpatialT/website/staging/data_v3_release/drug_screen/drug_enrichment/DS4A/DS4A.1/'+cell+'_POS_es.csv', index=False)

In [77]:
ds4a_es_dir = os.listdir('/share/fsmresfiles/SpatialT/cmap2_l1000/processed/6hr_only/enrich_ds4a/')
ds4a_es_dir

['ds4a_cmap2_6hr_b_enrich.csv',
 'ds4a_cmap2_6hr_othert_enrich.csv',
 'ds4a_cmap2_6hr_mono_enrich.csv',
 'ds4a_cmap2_6hr_mal_enrich.csv',
 'ds4a_cmap2_6hr_nk_enrich.csv',
 'ds4a_cmap2_6hr_fib_enrich.csv',
 'ds4a_cmap2_6hr_tmk_enrich.csv',
 'ds4a_cmap2_6hr_cd8_enrich.csv',
 'ds4a_cmap2_6hr_dc_enrich.csv',
 'ds4a_cmap2_6hr_endo_enrich.csv',
 'ds4a_cmap2_6hr_cd4_enrich.csv']

In [78]:
format_enrich(ds4a_es_dir[9], 'Endothelial')

In [80]:
# below result is for malignant, but endo should follow a similar format
# mal_inv = pd.read_csv('/share/fsmresfiles/SpatialT/website/staging/data_v3_release/drug_screen/drug_enrichment/DS4A/DS4A.1/Malignant_INV_es.csv')
# mal_inv[:50]

,CMAP_instance,Drug,Enrichment_score,P-value,Adj-p
0,PAC069_U2OS_6H:BRD-K05402890-001-08-4:20,BRD-K05402890,0.463,0.0,0.0
1,AML001_HL60_6H:BRD-K19706299:1.11111,MRE-269,0.422,0.0,0.0
2,CPC014_HT29_6H:BRD-K13514097-001-01-2:10,everolimus,0.407,0.0,0.0
3,CPC014_VCAP_6H:BRD-K13514097-001-01-2:10,everolimus,0.407,0.0,0.0
4,CPC006_CL34_6H:BRD-K23363278-001-01-3:1,CYT-997,0.405,0.0,0.0
5,CPC006_NCIH508_6H:BRD-A36275421-001-11-1:10,BRD-A36275421,0.403,0.0,0.0
6,CPC014_VCAP_6H:BRD-U86922168-000-01-3:10,QL-XII-47,0.402,0.0,0.0
7,DOS023_VCAP_6H:BRD-K07660364-001-01-7:4.91,BRD-K07660364,0.400,0.0,0.0
8,PAC031_U2OS_6H:BRD-K55127134-300-07-0:20,fluphenazine,0.395,0.0,0.0
9,CPC009_A375_6H:BRD-K58618158-001-01-3:10,BRD-K58618158,0.394,0.0,0.0


# CMAP2 Drug Perturbation Network (one sample, all cell types, all enriched drugs)

In [81]:
# cmap2 perturbation zscore
z_score = np.load('/share/fsmresfiles/SpatialT/cmap2_l1000/processed/6hr_only/cmap2_6hr_pert_z.npy')

In [83]:
z_score_df = pd.DataFrame(z_score)

In [85]:
z_score_df.columns = cmap2_cmap1_perturb_rank_dur.columns
z_score_df.index = cmap2_cmap1_perturb_rank_dur.index

In [86]:
z_score_df

,BIX-01338--LPROT001_A375_6H:BRD-K26863634:10,compound-e--LPROT002_A375_6H:BRD-A55453956:0.1,UNC-1215--LPROT004_YAPC_6H:BRD-K83189926:1,BAFILOMYCIN-A1--LPROT002_NPC.TAK_6H:BRD-K86787118:0.05,gatifloxacin--CPD003_MCF7_6H:BRD-A74980173-001-08-5:10,alfaxalone--CPD003_PC3_6H:J05,camptothecin--RAD001_MCF7_6H:BRD-K37890730-001-10-2:10,BRD-A72498772--AML001_PC3_6H:BRD-A72498772:0.37037,losmapimod--LPROT005_A549_6H:BRD-K56195681:0.25,BRD-K54343811--HDAC002_MCF7_6H:BRD-K54343811-001-01-1:10,...,bisphenol-a--CRCGN004_MCF7_6H:BRD-K41494493-001-07-0:10,DMSO--CRCGN001_HA1E_6H:A17,BRD-K45330754--CRCGN004_PC3_6H:BRD-K45330754-001-12-6:19.9,BRD-A21490874--CRCGN004_HA1E_6H:BRD-A21490874-001-01-6:20.1,estriol--CRCGN004_HA1E_6H:BRD-A88939772-001-06-4:20.1,ifosfamide--CRCGN004_HA1E_6H:BRD-A67097164-001-14-6:20,methyl-testosterone--CRCGN004_MCF7_6H:BRD-A08062015-001-01-0:20.8,busulfan--CRCGN004_MCF7_6H:BRD-K23204545-001-12-3:20.1,urethane--CRCGN001_HA1E_6H:BRD-K24297741-001-04-5:21.1,BRD-K45330754--CRCGN004_HMELZ_6H:BRD-K45330754-001-12-6:19.9
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
NAT2,-0.346811,-0.510760,0.653581,-0.219113,-0.8130,-0.7076,-0.5980,4.7780,0.229538,0.95075,...,-0.2775,-4.0908,1.6114,0.4712,0.3888,0.1507,-0.5814,0.4383,-0.073392,1.3709
ADA,-1.385003,-1.141117,0.367621,0.470731,-1.8020,0.3073,0.8671,5.0708,0.035623,0.37130,...,0.1709,0.7490,1.2741,-0.1040,-0.7910,0.9114,-0.3746,0.8406,-0.865260,0.8347
CDH2,1.235623,0.270382,0.703302,0.196277,0.1344,-0.9476,-0.2690,7.2086,-0.046486,-0.24965,...,0.1238,1.1624,-1.2506,0.4676,1.0785,-0.3713,0.7341,-0.4468,0.278728,-1.0679
AKT3,-0.418371,-1.084571,0.278982,0.085370,-2.2877,0.6352,1.0628,-2.7238,0.267737,0.28230,...,-0.0714,0.0000,0.6106,0.2030,-0.4291,-0.6277,-0.8892,0.6600,-0.619617,0.6312
MED6,0.114198,-0.115893,0.683868,0.440884,1.4054,0.3430,-2.8897,-5.1331,-0.070097,-0.86680,...,0.2998,1.6490,-0.4175,0.6970,1.0863,-0.4698,0.3756,-0.2987,0.062952,-1.7573
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SLC12A6,-0.601346,0.140269,0.907574,-0.182292,-0.4205,1.4701,2.2564,0.1856,-0.026107,0.28605,...,0.0732,0.1881,2.1246,0.0915,0.1923,0.4422,-1.0909,0.4469,-0.584766,-0.0641
PTBP3,-0.290443,-0.779189,0.694024,1.657170,1.3146,-1.0456,-0.0196,-0.7862,-0.419406,0.01615,...,0.0807,-1.6008,0.7801,0.6711,1.3835,-0.4211,1.0009,0.1602,-0.557091,-1.2701
KCNE2,-0.857904,-0.128861,1.107544,-0.853450,-1.4072,-0.0289,1.6940,7.8888,0.684473,0.43890,...,-0.2646,-1.4275,1.3316,-0.4992,-1.0856,0.1016,-0.9541,0.0955,-1.175276,2.4234


In [114]:
ds4a_deg = os.listdir('/share/fsmresfiles/SpatialT/drug-target/DS4A/DS4A.1/DGE_dec/')

In [110]:
# for all enriched drugs for a sample, retrieve cmap perturbations scores for all up and down degs of the sample
def get_perturb(cell_type):    
    dge = pd.read_csv('/share/fsmresfiles/SpatialT/drug-target/DS4A/DS4A.1/DGE_dec/'+cell_type+'.txt',sep='\t')
    dge_up = dge[(dge['stat']>0.5) & (dge['qval']<0.05) ]
    dge_down = dge[(dge['stat']<-0.5) & (dge['qval']<0.05) ]
    dge =pd.concat([dge_up, dge_down]).sort_values('stat')
    cell_type_deg_rank = cmap2_cmap1_perturb_rank_dur[cmap2_cmap1_perturb_rank_dur.index.isin(dge['gene'].tolist())]
    cell_type_deg_z = z_score_df[z_score_df.index.isin(dge['gene'].tolist())]
    es_inv = pd.read_csv('/share/fsmresfiles/SpatialT/website/staging/data_v3_release/drug_screen/drug_enrichment/DS4A/DS4A.1/'+cell_type+'_INV_es.csv')
    es_pos = pd.read_csv('/share/fsmresfiles/SpatialT/website/staging/data_v3_release/drug_screen/drug_enrichment/DS4A/DS4A.1/'+cell_type+'_POS_es.csv')
    for j,i in es_inv.iterrows():
        drug_idx = i['CMAP_instance']
        drug = i['Drug']
        drug_new_idx = drug + '--' +drug_idx
        print(j, drug_new_idx)
        zs = cell_type_deg_z[[drug_new_idx]]
        zs.columns = ['z']
        zs = zs.sort_values('z')
        rs = cell_type_deg_rank[[drug_new_idx]]
        rs=rs[rs.index.isin(zs.index)]
        rs.columns = ['rank']
        zs = zs.merge(rs, how = 'inner', left_index=True, right_index= True)
        zs = zs.merge(dge, how = 'left', left_index=True, right_on = 'gene')
        perturb_dir = '/share/fsmresfiles/SpatialT/cmap2_l1000/processed/6hr_only/perturb_ds4a/'+cell_type+'/'+'INV/'
        if not os.path.exists(perturb_dir):
            os.makedirs(perturb_dir)
        zs.to_csv(perturb_dir+drug_new_idx+'.csv', index =False)
    # same for pos direction
    for j,i in es_pos.iterrows():
        drug_idx = i['CMAP_instance']
        drug = i['Drug']
        drug_new_idx = drug + '--' +drug_idx
        print(j, drug_new_idx)
        zs = cell_type_deg_z[[drug_new_idx]]
        zs.columns = ['z']
        zs = zs.sort_values('z')
        rs = cell_type_deg_rank[[drug_new_idx]]
        rs=rs[rs.index.isin(zs.index)]
        rs.columns = ['rank']
        zs = zs.merge(rs, how = 'inner', left_index=True, right_index= True)
        zs = zs.merge(dge, how = 'left', left_index=True, right_on = 'gene')
        perturb_dir = '/share/fsmresfiles/SpatialT/cmap2_l1000/processed/6hr_only/perturb_ds4a/'+cell_type+'/'+'POS/'
        if not os.path.exists(perturb_dir):
            os.makedirs(perturb_dir)
        zs.to_csv(perturb_dir+drug_new_idx+'.csv', index =False)

In [125]:
ds4a_deg

['Endothelial.txt',
 'DC.txt',
 'Other.T.cell.txt',
 'B.cell.txt',
 'Malignant.txt',
 'TMKI67.txt',
 'Monocyte.txt',
 'CD4.txt',
 'Macrophage.txt',
 'Fibroblast.txt',
 'NK.txt',
 'CD8.txt']

In [128]:
for i in ds4a_deg:
    cell_type = i.split('.txt')[0]
    print(cell_type)
    get_perturb(cell_type)

Macrophage
0 BRD-K24514314--PAC039_U2OS_6H:BRD-K24514314-001-01-6:10.0068
1 BRD-K22719646--PAC045_U2OS_6H:BRD-K22719646-001-01-2:10.0741
2 BRD-K86268010--PAC038_U2OS_6H:BRD-K86268010-001-01-1:10.0364
3 BRD-K43271856--PAC038_U2OS_6H:BRD-K43271856-001-01-4:10.0219
4 BRD-K88720906--PAC038_U2OS_6H:BRD-K88720906-001-01-6:10.1273
5 BRD-K89650689--PAC066_U2OS_6H:BRD-K89650689-001-06-6:10
6 BRD-K28314528--PAC038_U2OS_6H:BRD-K28314528-001-01-9:9.89147
7 genistein--ERG005_VCAP_6H:BRD-K43797669-001-08-0:5
8 BRD-K09699976--PAC018_U2OS_6H:BRD-K09699976-001-01-3:9.99779
9 BRD-K64748422--PAC035_U2OS_6H:BRD-K64748422-001-01-0:10.1059
10 BRD-K82462389--PAC045_U2OS_6H:BRD-K82462389-001-01-7:10.0985
11 BRD-K24020784--PAC038_U2OS_6H:BRD-K24020784-001-01-5:9.93376
12 BRD-K03376048--PAC018_U2OS_6H:BRD-K03376048-001-01-8:9.99171
13 BRD-K59045095--PAC035_U2OS_6H:BRD-K59045095-001-01-4:10.0321
14 BRD-K10693654--FIBR007_MCLF051CN_6H:BRD-K10693654:40
15 BRD-K68844146--PAC057_U2OS_6H:BRD-K68844146-001-01-8:10.018

136 BRD-K79779816--CPC013_VCAP_6H:BRD-K79779816-001-01-2:10
137 BRD-K57490754--PAC052_U2OS_6H:BRD-K57490754-001-01-6:10.1514
138 BRD-K15619703--PAC013_U2OS_6H:BRD-K15619703-001-01-3:10.0162
139 BRD-K54023716--PAC011_U2OS_6H:BRD-K54023716-001-01-5:9.85259
140 BRD-M94053246--PAC056_U2OS_6H:BRD-M94053246-001-01-1:9.98674
141 mibefradil--ERG011_VCAP_6H:BRD-K09549677-300-02-6:2
142 BRD-K82206012--CPC014_HA1E_6H:BRD-K82206012-001-01-3:10
143 BRD-K09689420--PAC018_U2OS_6H:BRD-K09689420-001-01-6:10.2471
144 etomoxir--FIBR040_MCLF056CN_6H:BRD-K32405725-001-01-0:4
145 BRD-K77520720--PAC035_U2OS_6H:BRD-K77520720-001-01-7:10.1056
146 BRD-K87193132--PAC030_U2OS_6H:BRD-K87193132-001-01-3:10.0847
147 BRD-K11957311--ERG014_VCAP_6H:BRD-K11957311-001-01-5:10.0643
148 BRD-M85914958--PAC056_U2OS_6H:BRD-M85914958-001-01-3:10.0541
149 BRD-K24479270--PAC039_U2OS_6H:BRD-K24479270-001-01-4:10.0459
150 BRD-K18749194--CPC010_VCAP_6H:BRD-K18749194-001-03-7:10
151 BRD-K94843767--PAC046_U2OS_6H:BRD-K94843767-001-01

76 BRD-K28812281--PAC042_U2OS_6H:BRD-K28812281-001-01-9:10.1154
77 CGP-71683--CPC015_HT29_6H:BRD-K31912990-003-01-0:10
78 BRD-K03568952--DOS025_VCAP_6H:BRD-K03568952-001-01-4:5.04
79 tretinoin--AML001_HL60_6H:BRD-K71879491:10
80 wortmannin--PAC018_U2OS_6H:BRD-A75409952-001-02-4:20
81 BRD-K78845122--PAC056_U2OS_6H:BRD-K78845122-001-01-1:9.98658
82 BRD-K82863660--PAC007_U2OS_6H:BRD-K82863660-001-01-6:10.0602
83 fluphenazine--PAC045_U2OS_6H:BRD-K55127134-300-07-0:20
84 BRD-K47681666--PAC057_U2OS_6H:BRD-K47681666-001-01-0:10.0181
85 trifluoperazine--PAC041_U2OS_6H:BRD-K89732114-300-07-1:20
86 BRD-K65913831--PAC034_U2OS_6H:BRD-K65913831-001-01-4:10.0415
87 BRD-K90004145--PAC056_U2OS_6H:BRD-K90004145-001-01-2:9.9708
88 BRD-A19037878--PAC003_U2OS_6H:BRD-A19037878-001-01-5:20
89 BRD-K38557728--PAC008_U2OS_6H:BRD-K38557728-001-01-9:10.2817
90 thioridazine--PAC046_U2OS_6H:BRD-A84481105-003-18-0:20
91 trifluoperazine--PAC043_U2OS_6H:BRD-K89732114-300-07-1:20
92 BRD-K64937432--PAC056_U2OS_6H:BRD-K

16 BRD-K19216856--FIBR005_MCLF033SZ_6H:BRD-K19216856-001-01-9:4
17 CD-1530--FIBR023_MCLF126CN_6H:BRD-K25737009-001-02-0:4
18 BRD-K95606637--FIBR040_MCLF056CN_6H:BRD-K95606637-001-04-8:4
19 GW-9662--FIBR037_MCLF035SZ_6H:BRD-K93258693-001-05-8:4
20 ASN-05257430--FIBR004_MCLF027CN_6H:BRD-K44241590-001-03-3:4
21 glibenclamide--FIBR038_MCLF035SZ_6H:BRD-K36927236-001-17-7:4
22 BRD-K84924563--FIBR023_MCLF126CN_6H:BRD-K84924563-001-01-2:4
23 ASN-05257430--FIBR021_MCLF037SZ_6H:BRD-K44241590-001-03-3:4
24 fluoropyruvate--FIBR017_MCLF076SZ_6H:BRD-K01779529-236-01-5:4
25 glibenclamide--FIBR022_MCLF037SZ_6H:BRD-K36927236-001-17-7:4
26 BRD-K09398529--FIBR027_MCLF130CN_6H:BRD-K09398529-001-01-4:4
27 TGX-115--FIBR018_MCLF076SZ_6H:BRD-K67075780-001-03-7:4
28 CCT-031374--CPC006_CL34_6H:BRD-K90999434-003-01-9:10
29 carbamazepine--FIBR040_MCLF056CN_6H:BRD-K71799949-001-20-8:4
30 KIN001-244--LJP001_MCF7_6H:BRD-K09186807-001-02-4:0.4
31 BRD-K10882151--FIBR019_MCLF135CN_6H:BRD-K10882151-001-06-8:4
32 tranylc

153 BRD-K83354763--CPC006_CL34_6H:BRD-K83354763-001-01-1:6
154 BRD-K08447949--CPC008_HT29_6H:BRD-K08447949-001-01-2:10
155 BRD-A43640821--CPC009_VCAP_6H:BRD-A43640821-001-04-8:10
156 indirubin--CPC005_HT29_6H:BRD-K17894950-001-03-6:10
157 APO-866--FIBR021_MCLF037SZ_6H:BRD-K85563610-001-01-0:0.08
158 BRD-K08703257--FIBR023_MCLF126CN_6H:BRD-K08703257-001-06-3:4
159 minoxidil--CPC006_RKO_6H:BRD-K14888893-001-05-6:10
160 BRD-K51556300--KMS002_THP1_6H:BRD-K51556300-001-01-9:0
161 barasertib-HQPA--CPC014_MCF7_6H:E23
162 BRD-K16051505--PAC062_U2OS_6H:BRD-K16051505-001-01-8:20
163 RS-23597-190--FIBR010_MCLF123SZ_6H:BRD-K01868942-001-02-1:4
164 BRD-K84924563--FIBR038_MCLF035SZ_6H:BRD-K84924563-001-01-2:4
165 BRD-K41924482--CPC012_MCF7_6H:BRD-K41924482-001-01-2:10
166 BRD-K59939374--CPC014_HT29_6H:BRD-K59939374-001-01-0:10
167 estradiol--FIBR028_MCLF130CN_6H:BRD-K18910433:0.4
168 BRD-K51759637--PAC059_U2OS_6H:BRD-K51759637-001-01-3:9.99966
169 dexamethasone--CPC009_HT29_6H:BRD-A35108200-001-11-2

...

5 RAN-14A--PAC062_U2OS_6H:BRD-K36977165-001-01-1:20
6 AG-99--CPC005_A375_6H:BRD-K51223576-001-01-3:10
7 BRD-K95606637--FIBR005_MCLF033SZ_6H:BRD-K95606637-001-04-8:4
8 BRD-K58912123--ERG015_PC3_6H:BRD-K58912123-001-01-2:10.0309
9 BRD-A15079084--HOG001_MCF7_6H:BRD-A15079084-001-02-9:0.0045
10 narciclasine--CPC006_HCC15_6H:A15
11 benzamil--CPC002_HA1E_6H:BRD-K87158025-003-01-2:10
12 narciclasine--HOG003_A549_6H:BRD-K06792661-001-01-9:0.3704
13 K784-3188--PAC069_U2OS_6H:BRD-A72180425-001-09-8:20
14 BRD-K23412959--CPC008_HEPG2_6H:BRD-K23412959-001-05-5:10
15 anisomycin--CPC016_A549_6H:BRD-K91370081-001-10-3:10
16 BRD-A38479395--KTOX002_HPTEC_6H:BRD-A38479395:31.6
17 AGK-2--FIBR039_MCLF056CN_6H:BRD-K32536677-001-01-0:4
18 BRD-K95985487--KMS002_NOMO1_6H:BRD-K95985487-001-02-1:0.12
19 BRD-K10211218--PAC066_U2OS_6H:BRD-K10211218-001-05-3:10
20 bortezomib--ASG003_MCF7_6H:K09
21 BRD-K20616644--ERG015_PC3_6H:BRD-K20616644-001-01-4:10.0522
22 BI-2536--FIBR039_MCLF056CN_6H:BRD-K64890080-001-03-9:0.0

144 MW-STK33-1C--KMS002_NOMO1_6H:BRD-K38340366-001-12-4:1.11
145 BRD-K68946329--FIBR038_MCLF035SZ_6H:BRD-K68946329-001-02-8:4
146 anisomycin--CPC016_A375_6H:BRD-K91370081-001-10-3:10
147 ingenol--HOG001_MCF7_6H:BRD-A52650764-001-01-0:3.33
148 narciclasine--FIBR017_MCLF076SZ_6H:BRD-K06792661-001-01-9:0.4
149 BRD-K92317137--CPC006_HCC15_6H:L07
150 narciclasine--FIBR012_MCLF015CN_6H:BRD-K06792661-001-01-9:0.4
151 BRD-K62885083--PAC015_U2OS_6H:BRD-K62885083-001-01-2:9.98344
152 narciclasine--FIBR032_MCLF054CN_6H:BRD-K06792661-001-01-9:0.4
153 BRD-K01614657--CPC012_A375_6H:E09
154 narciclasine--FIBR011_MCLF015CN_6H:BRD-K06792661-001-01-9:0.4
155 GANT-61--CPC013_HCC515_6H:BRD-K09485525-001-01-3:10
156 BRD-K68946329--FIBR012_MCLF015CN_6H:BRD-K68946329-001-02-8:4
157 emetine--FIBR024_MCLF126CN_6H:BRD-K01976263-003-04-5:0.08
158 BI-2536--FIBR009_MCLF123SZ_6H:BRD-K64890080-001-03-9:0.08
159 emetine--CPC006_A375_6H:N07
160 BRD-K73261812--FIBR029_MCLF040SZ_6H:BRD-K73261812-001-01-1:4
161 ouabain--

86 BRD-K41597316--DOS015_VCAP_6H:BRD-K41597316-001-01-8:4.97
87 BRD-K32475622--PAC049_U2OS_6H:BRD-K32475622-001-01-7:9.99379
88 BRD-K48406740--PAC067_U2OS_6H:BRD-K48406740-001-05-2:10
89 mephentermine--CPC004_PC3_6H:BRD-K18194590-065-05-6:10
90 BRD-K57490754--PAC052_U2OS_6H:BRD-K57490754-001-01-6:10.1514
91 temozolomide--ASG003_MCF7_6H:I12
92 IWP-2--FIBR015_MCLF022CN_6H:BRD-K82924899-001-02-7:4
93 BRD-K98239824--PAC032_U2OS_6H:BRD-K98239824-001-01-4:10.0055
94 BRD-K34710061--PAC027_U2OS_6H:BRD-K34710061-001-01-3:9.69909
95 BRD-K51790412--PAC010_U2OS_6H:BRD-K51790412-001-01-9:10.0981
96 BRD-K58977196--PAC016_U2OS_6H:BRD-K58977196-001-01-7:9.74349
97 BRD-K96893659--PAC065_U2OS_6H:BRD-K96893659-001-04-4:10
98 megestrol-acetate--CPC005_HT29_6H:BRD-K19507340-001-03-1:10
99 clopidogrel--PAC005_U2OS_6H:BRD-A93269281-003-01-0:10
100 BRD-K93972090--PAC022_U2OS_6H:BRD-K93972090-001-01-2:10.4041
101 BRD-K64581877--PAC010_U2OS_6H:BRD-K64581877-001-01-5:10.1341
102 BRD-A56831909--CPC007_HT29_6H:BRD

27 BRD-K36596997--DOS030_VCAP_6H:BRD-K36596997-001-01-3:5.03
28 BRD-K97277752--DOS011_VCAP_6H:BRD-K97277752-001-01-1:5
29 BRD-K03352392--PAC018_U2OS_6H:BRD-K03352392-001-01-1:9.93424
30 BRD-K74348865--CPC008_HCC515_6H:BRD-K74348865-001-04-5:10
31 BRD-K25577594--PAC047_U2OS_6H:BRD-K25577594-001-01-3:10.0374
32 saclofen--CPC002_PC3_6H:BRD-A24122750-001-02-9:10
33 ibrutinib--EMU001_MINO_6H:P01
34 BRD-K08247496--PAC068_U2OS_6H:BRD-K08247496-001-06-2:10
35 emetine--FIBR023_MCLF126CN_6H:BRD-K01976263-003-04-5:0.08
36 fluperlapine--CPC003_HA1E_6H:BRD-K15715913-001-01-5:10
37 anguidine--ASG003_U2OS_6H:I16
38 BRD-K37407046--PAC023_U2OS_6H:BRD-K37407046-001-01-3:10.0909
39 BRD-K28351377--PAC009_U2OS_6H:BRD-K28351377-001-01-3:10.1621
40 BRD-K28184331--PAC017_U2OS_6H:BRD-K28184331-001-01-3:10.0369
41 BRD-K56125296--PAC028_U2OS_6H:BRD-K56125296-001-01-4:10.0115
42 YK-4279--FIBR028_MCLF130CN_6H:BRD-A62182663:0.8
43 BRD-K95153342--PAC010_U2OS_6H:BRD-K95153342-001-01-3:10.0177
44 blebbistatin--CPC006_

170 BRD-K49872883--PAC016_U2OS_6H:BRD-K49872883-001-01-0:10.0513
171 BRD-K52012504--PAC047_U2OS_6H:BRD-K52012504-001-01-6:10.0374
172 BRD-A32449311--CPC010_PC3_6H:BRD-A32449311-001-05-1:10
173 nilotinib--EMU001_TMD8_6H:D11
174 BRD-K03070790--PAC018_U2OS_6H:BRD-K03070790-001-01-7:9.95934
175 AG-99--CPC005_VCAP_6H:BRD-K51223576-001-01-3:10
176 iocetamic-acid--PAC006_U2OS_6H:BRD-A37492983-001-03-8:3.25756
177 BRD-K81586131--PAC010_U2OS_6H:BRD-K81586131-001-01-5:9.5356
178 BRD-K70399291--ERG010_VCAP_6H:BRD-K70399291-001-01-2:10.083
179 GANT-61--CPC013_HCC515_6H:BRD-K09485525-001-01-3:10
180 BRD-K28480589--PAC014_U2OS_6H:BRD-K28480589-001-01-4:10.1717
181 BRD-K87276639--PAC012_U2OS_6H:BRD-K87276639-001-01-4:10.0758
182 BRD-K51061297--PAC016_U2OS_6H:BRD-K51061297-001-01-5:9.98065
183 BRD-K08521815--PAC019_U2OS_6H:BRD-K08521815-001-01-5:9.97275
184 baccatin-III--CPC002_PC3_6H:BRD-A84189516-001-01-2:10
185 BRD-K31896531--PAC027_U2OS_6H:BRD-K31896531-001-01-5:10.0031
186 methylparaben--KTOX002_

In [129]:
# sample perturbation file for a cell type's enriched drug
sample_cmap2 = pd.read_csv('/share/fsmresfiles/SpatialT/cmap2_l1000/processed/6hr_only/perturb_ds4a/Macrophage/INV/BRD-K24514314--PAC039_U2OS_6H:BRD-K24514314-001-01-6:10.0068.csv')

In [130]:
sample_cmap2

,z,rank,gene,stat,pval,qval
0,-6.8631,1.0,SLC11A2,0.518406,1.191595e-143,5.523042e-140
1,-2.6814,5.0,ESRP1,-1.298291,0.000000e+00,0.000000e+00
2,-2.5818,7.0,FPR3,2.404700,0.000000e+00,0.000000e+00
3,-2.4656,13.0,CEBPA,1.997606,0.000000e+00,0.000000e+00
4,-2.4490,15.0,GPNMB,2.913453,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...
1411,2.1002,12310.0,PLCG1,-0.614794,2.334339e-286,1.081966e-282
1412,2.1814,12315.0,CSE1L,-0.618755,1.010225e-240,4.682393e-237
1413,2.5111,12323.0,GTPBP4,-0.647590,6.436819e-251,2.983465e-247
1414,2.5223,12324.0,EZR,-1.244729,0.000000e+00,0.000000e+00


In [181]:
# format pertrbation network to create edge and node file
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (-2,2))

perturb_dir = ('/share/fsmresfiles/SpatialT/cmap2_l1000/processed/6hr_only/perturb_ds4a/')
os.chdir(perturb_dir)
for cell in os.listdir(perturb_dir):
    for direc in os.listdir(perturb_dir+cell+'/'):
        os.chdir(perturb_dir+cell+'/'+direc+'/')
        for perturb in os.listdir():
            print(cell, direc, perturb)
            drug = perturb.split('--')[0]
            cmap_instance = perturb.split('--')[1].split('.csv')[0]
            cmap_drug = perturb.split('.csv')[0]
            perturb_df = pd.read_csv(perturb)
            deg_gene_num = perturb_df.shape[0]
            perturb_edges = pd.DataFrame()
            perturb_edges['Source'] = [drug for i in range(deg_gene_num)]
            perturb_edges['CMAP_instance'] = [cmap_instance for i in range(deg_gene_num)]
            perturb_edges['Target'] = perturb_df['gene'].tolist()
            perturb_edges['log2fc'] = perturb_df['stat'].tolist()
            perturb_edges['exp_z'] = perturb_df['z'].tolist()
            perturb_edges['exp_z_norm'] = [i[0] for i in scaler.fit_transform(np.array(perturb_edges['exp_z'].tolist()).reshape(-1, 1))]
            perturb_edges['abs_log2fc'] = [abs(i) for i in perturb_df['stat'].tolist()]
            perturb_edges['sign_log2fc'] = [np.sign(i) for i in perturb_df['stat'].tolist()]
            perturb_edges = perturb_edges.sort_values('exp_z')
            head_edges = perturb_edges.head(30)
            tail_edges = perturb_edges.tail(30)
            perturb_edges_sub = pd.concat([head_edges, tail_edges])
            
            perturb_nodes = pd.DataFrame()
            perturb_nodes['Id'] = perturb_edges_sub['Target'] 
            perturb_nodes['Label'] = perturb_edges_sub['Target'] 
            perturb_nodes['log2fc'] = perturb_edges_sub['log2fc']
            perturb_nodes['sign_log2fc'] = perturb_edges_sub['sign_log2fc'] 
            perturb_nodes['abs_log2fc'] = perturb_edges_sub['abs_log2fc'] 
            drug_row = pd.DataFrame({'Id':[perturb_edges_sub['Source'].unique().tolist()[0]],
                                 'Label':[perturb_edges_sub['Source'].unique().tolist()[0]],
                                 'log2fc':[10],
                                 'sign_log2fc':[0],
                                 'abs_log2fc':[10]})
            perturb_nodes = pd.concat([drug_row, perturb_nodes])
            
            perturb_sample_dir = '/share/fsmresfiles/SpatialT/website/staging/data_v3_release/drug_screen/drug_perturbation/DS4A/DS4A.1/'+cell+'/'+direc+'/'
            if not os.path.exists(perturb_sample_dir):
                os.makedirs(perturb_sample_dir)

            perturb_edges_sub.to_csv(perturb_sample_dir+cmap_drug+'_perturb.csv',index=False)
            perturb_nodes.to_csv(perturb_sample_dir+cmap_drug+'_perturb_nodes.csv',index=False)
    

Endothelial POS AZD-7545--FIBR040_MCLF056CN_6H:BRD-K52836380-001-01-7:4.csv
Endothelial POS VER-155008--FIBR016_MCLF022CN_6H:BRD-K32330832-001-01-0:4.csv
Endothelial POS BRD-K26510616--HDAC002_PC3_6H:BRD-K26510616-001-01-7:5.csv
Endothelial POS BRD-K51654114--PAC028_U2OS_6H:BRD-K51654114-001-01-8:10.0025.csv
Endothelial POS BRD-K91194423--DOS013_VCAP_6H:BRD-K91194423-001-01-3:4.98.csv
Endothelial POS LDN-193189--CPC006_LOVO_6H:BRD-K04853698-003-01-4:10.csv
Endothelial POS KIN001-127--CPC014_HT29_6H:BRD-A29901043-001-01-0:10.csv
Endothelial POS NU-1025--CPC006_NOMO1_6H:BRD-K48692744-001-01-2:10.csv
Endothelial POS BRD-K60804903--PAC047_U2OS_6H:BRD-K60804903-001-01-3:10.1051.csv
Endothelial POS BRD-K72161690--PAC015_U2OS_6H:BRD-K72161690-001-01-1:9.98693.csv
Endothelial POS VU-0418000-1--CPC008_HEPG2_6H:BRD-K97516968-001-01-2:10.csv
Endothelial POS BRD-K64558129--PAC013_U2OS_6H:BRD-K64558129-001-01-9:10.0192.csv
Endothelial POS BRD-K93260425--PAC032_U2OS_6H:BRD-K93260425-001-01-8:10.0398

Endothelial POS BRD-K31337291--PAC017_U2OS_6H:BRD-K31337291-001-01-0:10.0439.csv
Endothelial POS DC-45-A2--CPC006_THP1_6H:J13.csv
Endothelial POS PAC-1--FIBR024_MCLF126CN_6H:BRD-K92991072-001-01-5:25.4781.csv
Endothelial POS BI-2536--FIBR018_MCLF076SZ_6H:BRD-K64890080-001-03-9:0.08.csv
Endothelial POS BRD-K14696368--CPC006_SW948_6H:N23.csv
Endothelial POS nutlin-3--CPC006_NOMO1_6H:BRD-A12230535-001-02-6:44.4.csv
Endothelial POS BRD-U29336476--CPC014_VCAP_6H:BRD-U29336476-000-01-3:10.csv
Endothelial POS nutlin-3--FIBR021_MCLF037SZ_6H:BRD-A12230535-001-01-8:4.csv
Endothelial POS BRD-K51520673--PAC016_U2OS_6H:BRD-K51520673-001-01-4:10.1141.csv
Endothelial POS BRD-K81538844--CPC010_HT29_6H:BRD-K81538844-001-06-3:10.csv
Endothelial POS BRD-K92932331--CPC010_VCAP_6H:BRD-K92932331-001-04-0:10.csv
Endothelial POS PHTPP--CPC001_VCAP_6H:M18.csv
Endothelial POS BRD-K45076286--PAC040_U2OS_6H:BRD-K45076286-001-01-5:10.0427.csv
Endothelial POS BRD-K82580504--CPC009_HT29_6H:BRD-K82580504-001-01-6:10.

Endothelial INV BRD-K62473229--PAC059_U2OS_6H:BRD-K62473229-001-01-2:9.93749.csv
Endothelial INV BRD-K90826279--FIBR019_MCLF135CN_6H:BRD-K90826279-001-03-2:4.csv
Endothelial INV tranylcypromine--FIBR039_MCLF056CN_6H:BRD-K47029922-003-03-4:4.csv
Endothelial INV teniposide--FIBR013_MCLF162SZ_6H:BRD-A35588707-001-02-2:0.3125.csv
Endothelial INV isradipine--FIBR036_MCLF137CN_6H:BRD-A90799790-001-04-1:4.csv
Endothelial INV molindone--CPC005_MCF7_6H:O10.csv
Endothelial INV imipramine--FIBR014_MCLF162SZ_6H:BRD-K38436528-003-15-4:4.csv
Endothelial INV BRD-K56285711--PAC016_U2OS_6H:BRD-K56285711-001-01-3:9.99437.csv
Endothelial INV isotretinoin--CPC014_HEPG2_6H:BRD-K76723084-001-15-8:10.csv
Endothelial INV inhibitor-BEC--FIBR001_MCLF117SZ_6H:BRD-K53792571-001-02-8:4.csv
Endothelial INV genistein--PAC011_U2OS_6H:BRD-K43797669-001-10-6:20.csv
Endothelial INV BRD-K32380923--DOS023_VCAP_6H:BRD-K32380923-001-01-5:4.92.csv
Endothelial INV selumetinib--FIBR030_MCLF040SZ_6H:BRD-K57080016-001-02-7:4.csv

Endothelial INV BRD-K06748700--CPC007_PC3_6H:BRD-K06748700-001-05-5:10.csv
Endothelial INV BRD-K32983008--PAC054_U2OS_6H:BRD-K32983008-001-01-4:10.0319.csv
Endothelial INV STK-721917--PCLB001_JURKAT_6H:H23.csv
Endothelial INV BRD-K91467861--PAC049_U2OS_6H:BRD-K91467861-001-01-2:10.1129.csv
Endothelial INV nitrendipine--CPC006_THP1_6H:E16.csv
Endothelial INV BRD-K03116744--PAC027_U2OS_6H:BRD-K03116744-001-01-5:10.012.csv
Endothelial INV necrostatin-1--FIBR004_MCLF027CN_6H:BRD-A36318220-001-02-1:4.csv
Endothelial INV GW-405833--FIBR033_MCLF100SZ_6H:BRD-K10705233-003-01-0:4.csv
Endothelial INV NU-1025--CPC006_THP1_6H:E03.csv
Endothelial INV BRD-K56525823--PAC016_U2OS_6H:BRD-K56525823-001-01-7:10.0511.csv
Endothelial INV BRD-K30633507--PAC010_U2OS_6H:BRD-K30633507-001-01-3:10.1992.csv
Endothelial INV BRD-K40853697--FIBR028_MCLF130CN_6H:BRD-K40853697:4.csv
Endothelial INV BRD-K40265664--PAC049_U2OS_6H:BRD-K40265664-001-01-1:10.0241.csv
Endothelial INV purmorphamine--CPC006_PL21_6H:BRD-K7339

NK POS digitoxin--ASG003_A549_6H:I07.csv
NK POS BRD-K19216856--CPC006_NOMO1_6H:BRD-K19216856-001-02-7:88.8.csv
NK POS emetine--CPC007_PC3_6H:BRD-K03067624-003-19-3:10.csv
NK POS anisomycin--CPC016_A375_6H:BRD-K91370081-001-10-3:10.csv
NK POS BRD-K38165718--PAC017_U2OS_6H:BRD-K38165718-001-01-3:10.038.csv
NK POS BRD-K62885083--PAC015_U2OS_6H:BRD-K62885083-001-01-2:9.98344.csv
NK POS BRD-K24932095--DOS025_VCAP_6H:BRD-K24932095-001-01-5:5.04.csv
NK POS BRD-K34311178--FIBR024_MCLF126CN_6H:BRD-K34311178-001-01-0:40.csv
NK POS BRD-K73261812--FIBR038_MCLF035SZ_6H:BRD-K73261812-001-01-1:4.csv
NK POS emetine--ASG003_U2OS_6H:J04.csv
NK POS narciclasine--CPC006_OV7_6H:BRD-K06792661-001-01-9:10.csv
NK POS BRD-K76674262--HOG002_A549_6H:BRD-K76674262-001-02-5:1.11.csv
NK POS BRD-K70489510--ERG015_PC3_6H:BRD-K70489510-001-01-6:10.0072.csv
NK POS narciclasine--CPC006_HCC15_6H:A15.csv
NK POS BRD-K68946329--FIBR012_MCLF015CN_6H:BRD-K68946329-001-02-8:4.csv
NK POS AGK-2--FIBR039_MCLF056CN_6H:BRD-K3253667

...

CD8 INV BRD-K27139040--PAC017_U2OS_6H:BRD-K27139040-001-01-0:10.111.csv
CD8 INV BRD-K54864501--PAC023_U2OS_6H:BRD-K54864501-001-02-9:9.99644.csv
CD8 INV BRD-K78716413--FIBR023_MCLF126CN_6H:BRD-K78716413-001-02-0:0.04.csv
CD8 INV SKI-II--PAC003_U2OS_6H:BRD-K20755323-001-01-8:10.csv
CD8 INV BRD-K30257736--PAC017_U2OS_6H:BRD-K30257736-001-01-4:10.0263.csv
CD8 INV procainamide--CPC020_MCF7_6H:BRD-K75089421-003-21-1:10.csv
CD8 INV lovastatin--FIBR020_MCLF135CN_6H:BRD-A70155556-001-03-6:4.csv
CD8 INV L-701252--PAC004_U2OS_6H:BRD-K10176267-001-01-7:10.csv
CD8 INV BRD-K27757818--PAC036_U2OS_6H:BRD-K27757818-001-01-9:10.0467.csv
CD8 INV BRD-K23406211--PAC056_U2OS_6H:BRD-K23406211-001-01-0:9.93426.csv
CD8 INV duloxetine--CPC015_HEPG2_6H:BRD-K71103788-003-04-7:10.csv
CD8 INV clopidogrel--PAC005_U2OS_6H:BRD-A93269281-003-01-0:10.csv
CD8 INV BRD-K42633304--PAC029_U2OS_6H:BRD-K42633304-001-01-4:10.0083.csv
CD8 INV BRD-K10917236--PAC018_U2OS_6H:BRD-K10917236-001-01-2:10.0104.csv
CD8 INV BRD-K50942788

CD8 INV manumycin-a--EMU001_MINO_6H:E07.csv
CD8 INV gefitinib--CPC014_HT29_6H:BRD-K64052750-001-05-0:10.csv
CD8 INV BRD-K68055156--PAC037_U2OS_6H:BRD-K68055156-001-01-8:9.93334.csv
CD8 INV BRD-K57490754--PAC052_U2OS_6H:BRD-K57490754-001-01-6:10.1514.csv
CD8 INV SA-247615--PAC008_U2OS_6H:BRD-K67870070-001-01-4:10.143.csv
CD8 INV BRD-K35049668--PAC021_U2OS_6H:BRD-K35049668-001-01-2:10.0849.csv
CD8 INV EGF--LJP003_MCF7_6H:CMAP-L097:0.1.csv
CD8 INV BRD-K11792293--PAC013_U2OS_6H:BRD-K11792293-001-01-8:10.0237.csv
CD8 INV BRD-K25690923--CPC006_RMUGS_6H:D01.csv
CD8 INV BRD-K12710356--PAC018_U2OS_6H:BRD-K12710356-001-01-3:9.9085.csv
CD8 INV BRD-K45056520--PAC016_U2OS_6H:BRD-K45056520-001-01-3:9.87382.csv
CD8 INV BRD-K68949590--PAC015_U2OS_6H:BRD-K68949590-001-01-6:9.90073.csv
CD8 INV BRD-K55482336--PAC012_U2OS_6H:BRD-K55482336-001-01-2:10.0159.csv
CD8 INV BRD-K51790412--PAC010_U2OS_6H:BRD-K51790412-001-01-9:10.0981.csv
CD8 INV tyrphostin-47--CPC017_A549_6H:BRD-K96778649-001-03-2:10.csv
CD8 INV

Other.T.cell POS BRD-K43069600--CPC007_HT29_6H:BRD-K43069600-001-03-7:10.csv
Other.T.cell POS BRD-K51185182--PAC016_U2OS_6H:BRD-K51185182-001-01-0:10.0608.csv
Other.T.cell POS BRD-K92579151--PAC025_U2OS_6H:BRD-K92579151-001-01-2:9.657.csv
Other.T.cell POS BRD-K87276639--PAC012_U2OS_6H:BRD-K87276639-001-01-4:10.0758.csv
Other.T.cell POS BRD-K33179234--PAC028_U2OS_6H:BRD-K33179234-001-01-4:10.0501.csv
Other.T.cell POS BRD-K64305210--PAC053_U2OS_6H:BRD-K64305210-004-01-9:10.0041.csv
Other.T.cell POS BRD-K07395346--CPC013_PC3_6H:BRD-K07395346-001-01-3:10.csv
Other.T.cell POS troglitazone--PAC016_U2OS_6H:BRD-A13084692-001-06-6:20.csv
Other.T.cell POS BRD-A15079084--HOG001_MCF7_6H:BRD-A15079084-001-02-9:0.0045.csv
Other.T.cell POS BRD-K13146924--PAC040_U2OS_6H:BRD-K13146924-001-01-2:10.1213.csv
Other.T.cell POS tetramethylsilane--CPC016_MCF7_6H:BRD-K45988865-001-01-0:10.csv
Other.T.cell POS BRD-K99149118--PAC058_U2OS_6H:BRD-K99149118-001-01-9:10.0579.csv
Other.T.cell POS SSR-69071--CPC018_MC

Other.T.cell POS simvastatin--CPC006_SKM1_6H:BRD-K22134346-001-09-0:10.csv
Other.T.cell POS PSH-010--CPC007_A375_6H:P05.csv
Other.T.cell POS BRD-K93023739--CPC006_CL34_6H:BRD-K93023739-001-01-1:6.csv
Other.T.cell POS pyroxamide--CPC013_MCF7_6H:BRD-K11663430-001-01-5:10.csv
Other.T.cell POS BRD-K73707305--CPC013_PC3_6H:BRD-K73707305-001-04-1:10.csv
Other.T.cell POS BRD-K36997940--PAC026_U2OS_6H:BRD-K36997940-001-01-1:10.0454.csv
Other.T.cell POS salicin--CPC016_A549_6H:BRD-K64614248-001-06-5:10.csv
Other.T.cell POS BRD-K62748060--PAC019_U2OS_6H:BRD-K62748060-001-01-9:10.0674.csv
Other.T.cell POS BRD-K22243404--PAC025_U2OS_6H:BRD-K22243404-001-01-5:10.0077.csv
Other.T.cell POS BRD-K96238295--PAC047_U2OS_6H:BRD-K96238295-001-01-8:10.1137.csv
Other.T.cell POS BRD-K69343050--PAC023_U2OS_6H:BRD-K69343050-001-02-2:10.0484.csv
Other.T.cell INV IWP-2--FIBR015_MCLF022CN_6H:BRD-K82924899-001-02-7:4.csv
Other.T.cell INV BRD-K73143528--CPC007_HT29_6H:BRD-K73143528-001-04-0:10.csv
Other.T.cell INV B

Other.T.cell INV ethotoin--CPC005_PC3_6H:BRD-A30205217-001-03-9:10.csv
Other.T.cell INV BRD-K72773309--PAC050_U2OS_6H:BRD-K72773309-001-01-6:9.95298.csv
Other.T.cell INV MW-SHH-151--PCLB001_A375_6H:BRD-K14700434-001-02-6:10.csv
Other.T.cell INV tanespimycin--HOG003_MCF7_6H:BRD-A61304759-001-01-0:0.0015.csv
Other.T.cell INV L-690330--CPC006_NCIH596_6H:BRD-K28470988-001-01-2:10.csv
Other.T.cell INV BRD-K74300740--PAC030_U2OS_6H:BRD-K74300740-001-01-3:10.0541.csv
Other.T.cell INV BRD-K98239824--PAC032_U2OS_6H:BRD-K98239824-001-01-4:10.0055.csv
Other.T.cell INV BRD-K23582869--CPC019_A375_6H:BRD-K23582869-001-01-3:10.csv
Other.T.cell INV BRD-K39824708--PAC033_U2OS_6H:BRD-K39824708-001-01-6:10.0478.csv
Other.T.cell INV BRD-K40959762--DOS017_VCAP_6H:BRD-K40959762-001-01-0:5.05.csv
Other.T.cell INV BRD-K29728947--PAC011_U2OS_6H:BRD-K29728947-001-01-2:10.2534.csv
Other.T.cell INV BRD-K28294686--PAC017_U2OS_6H:BRD-K28294686-001-01-1:9.97921.csv
Other.T.cell INV BRD-K95672449--PAC032_U2OS_6H:BRD-